# RAG On Local Database
In this part we use our prepared data for you: EngineeringHoistory3Books

Make sure you have all the `xxx_text.parquet` and `xxx_image.parquet` files ready.
This notebook demonstrates:
- how to perform vector search on local database
- perform augmented genAI based on searched results using Azure AI ChatGPT

## Setup
Import environment packages.

In [ ]:
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import List, Dict, Tuple
from IPython.display import Image

import os
import pandas as pd
import torch
import torch.nn.functional as F
import numpy as np

load_dotenv(override=True)

## Build Local Text and Image Databases

In [ ]:
textDB = None
imageDB = None
contentTensors = None
captionTensors= None
INPUT_PATH = None
TEXT_K = None
IMAGE_K = None

def initDB():
    global textDB, imageDB, contentTensors, captionTensors, INPUT_PATH, TEXT_K, IMAGE_K
    if textDB is None or imageDB is None or contentTensors is None or captionTensors is None or INPUT_PATH is None or TEXT_K is None or IMAGE_K is None:
        print("RAG reading database into memory")
        # IMPORTANT ************************  CONFIGURABLE VARIABLES
        INPUT_PATH = "EngineeringHistory3Books" #You can change this to the path of your database
        TEXT_K = 10
        IMAGE_K = 5
        # setup text database
        textDB = pd.read_parquet(INPUT_PATH + "_text.parquet", engine="pyarrow")
        # normalize text vectors
        contentTensors = F.normalize(torch.from_numpy(np.stack(textDB['contentVector'].to_numpy())), p=2, dim=1).to(torch.float32)
        # setup image database
        imageDB = pd.read_parquet(INPUT_PATH + "_image.parquet", engine="pyarrow")
        # normalize image caption vectors
        captionTensors = F.normalize(torch.from_numpy(np.stack(imageDB['captionVector'].to_numpy())), p=2, dim=1).to(torch.float32)
    print("RAG has finished reading database into memory")

In [ ]:
initDB()

## Vector Search & RAG

To perform RAG, we first Embed our query to perform vector search on our local databases. Then we pass the retrieved relevant resutls to Azure ChatGPT to generate desired answers.

In [ ]:
# embedding model variables
embedding_client = AzureOpenAI(
    api_key = os.getenv("EMBEDDING_OPENAI_API_KEY"),
    api_version = os.getenv("EMBEDDING_OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("EMBEDDING_OPENAI_API_ENDPOINT")
)
embedding_model = os.getenv("EMBEDDING_DEPLOYMENT_NAME")
# Gen AI variables
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
api_key = os.getenv("AZURE_OPENAI_API_KEY")  
deployment_name = 'trygpt4o'  
api_version = '2024-02-01'  # this might change in the future  
client = AzureOpenAI(  
    api_key=api_key,
    api_version=api_version,  
    base_url=f"{api_base}/openai/deployments/{deployment_name}"  
)

In [ ]:
def query(messages: List[Dict]) -> Tuple[str, List[Dict]]:
    # load global variables
    global api_base, api_key, api_version, deployment_name, client
    global embedding_client, embedding_model
    global textDB, imageDB, contentTensors, captionTensors, INPUT_PATH, TEXT_K, IMAGE_K

    # get latest message from user
    query = messages[-1]['content']

    # embed and normalize user query
    queryVector = embedding_client.embeddings.create(input=[query], model=embedding_model).data[0].embedding
    queryTensor = F.normalize(torch.tensor(queryVector), p=2, dim=0).to(torch.float32)

    # search for text in textDB
    text_cosine_similarities = torch.matmul(queryTensor, contentTensors.transpose(0,1))
    topk_text_indices = torch.topk(text_cosine_similarities, k=TEXT_K).indices
    print("-----------------------------------------------------")
    print("Top K text indices:")
    print(textDB.iloc[topk_text_indices.numpy()])
    text_search_results_for_GPT = str(textDB.iloc[topk_text_indices.numpy()][['id','content']].to_dict('records'))

    # search for image in imageDB (score = query cosine similarty with caption * k number of text extracts + for each text extract's cosine similarity with caption)
    image_search_score = torch.matmul(queryTensor, captionTensors.transpose(0,1)) * TEXT_K
    for index in topk_text_indices.numpy():
        image_search_score = image_search_score + torch.matmul(contentTensors[index], captionTensors.transpose(0,1))
    topk_image_indices = torch.topk(image_search_score, k=IMAGE_K).indices
    print("-----------------------------------------------------")
    print("Top K image indices:")
    print(imageDB.iloc[topk_image_indices.numpy()])
    image_search_results = imageDB.iloc[topk_image_indices.numpy()][['id', 'image', 'caption']].to_dict('records')
    image_search_results_for_GPT = str(imageDB.iloc[topk_image_indices.numpy()][['id','caption']].to_dict('records'))

    # Generate response
    messages[-1]['content']=f'''Question: {query}
    Sources: {text_search_results_for_GPT}
    
    Answer the question. Be specific in your answers. Answer ONLY with the facts listed in the list of sources above. If the question is not related to the sources, politely decline. If there isn't enough information from the sources, say you don't know. Do not generate answers that don't use the sources above. When you use information related to a particular source, include citation tags with the id as content like the example below. There can be multiple citation tags. Interleave images if it is relevant to your answer, relevancy can be determined from the provided captions. When you use images, include image tags with the id as content like the example below.
    Example:
    The Po Shan Road landslide incident occurred in 1972 and resulted in the deaths of 67 people. This landslide also affected part of the University of Hong Kong campus. The incident was one of two catastrophic landslides that year, which in total caused over 130 casualties and left more than 5,000 people homeless. The Po Shan Road landslide demolished a 12-storey apartment block. [citation:bookname_1_2]

    [image:bookname_3_4]
    
    This tragic event, along with the Sau Mau Ping landslide, raised serious concerns about public safety concerning hillside development. These disasters led to the formation of an International Review Panel, which included experts such as Professor Sean Mackey and Professor Peter Lumb. The panel made significant recommendations on the management of landslide risks in Hong Kong. [citation:bookname_5_6] [citation:bookname_7_8]
    '''
    completion = client.chat.completions.create(
        model = deployment_name,
        messages = messages
    )
    chat_response = completion.choices[0].message.content
    return(chat_response, image_search_results)

Now we define a variable to store chat history.

In [ ]:
chat_history = []

Change `user_prompt` to your own query and retrieve the response and images.

In [ ]:
user_prompt = "tell me what are things author present in the work" # Change your query here
chat_history.append({'role': 'user', 'content': user_prompt}) 
response, image_list = query(chat_history)
chat_history.append({'role' : 'assistant', 'content' : response})

The code below is for displaying chat history and retrieved images.\
User queries are in Green.\
Responses are in Red.

In [ ]:
for chat in chat_history:
    if (chat['role'] == 'assistant'):
        print('\x1b[6;31m' + chat['content'] + '\x1b[0m')
    elif (chat['role'] == 'user'):
        print('\x1b[6;32m' + chat['content'] + '\x1b[0m')
